# MD2SL - Master in Data Science and Statistical Learning

**Numerical Calculus and Linear Algebra**

Exercises 01: Introduction: errors and machine arithmetic

Deadline: 31/05/2024

In [ ]:
# Installing packages.
import numpy as np
import math

# Exercise 1
1.  Write a function which takes in input $x\in\mathbb{R}$ and $n\in\mathbb{N}$ and returns the Taylor polynomial approximation of $\sin(x)$ centered in $0$ of order $n$.
\begin{equation}
  T_n(x) = \sum_{k=0}^n\frac{(-1)^k}{(2k+1)!}x^{2k+1}.
\end{equation}

2.  Use the function of point (1) to compute $\sin(x)$ for $n=20$ at points $x=\frac{\pi}{2}, \frac{\pi}{2} + 2\cdot10^{16}\pi$. Comment the results.

Exercise 1.1 - Solution.

L'approssimazione di $\sin(x)$ con il suo polinomio di Taylor di ordine n sfrutta il fatto che
*   $\sin(0) = 0$
*   $\sin'(0) = \cos(0) = 1$
*   $\sin''(0) = -\sin(0) = 0$
*   $\sin'''(0) = -\cos(0) = -1$
*   $\sin^{(4)}(0) = \sin(0) = 0$

Poiché $\sin^{(4)}(x) = \sin(x)$, lo schema poi si ripete.


L'implementazione che segue stampa a video anche i valori degli elementi critici nel calcolo di
$p_n = \dfrac{x^n}{n!} = \underbrace{\dfrac{x^{n-1}}{(n-1)!}}_{p_{n-1}}\, \underbrace{\dfrac{x}{n}}_{v_n} = p_{n-1}\, v_{n}$

in termini della loro rappresentazione in virgola mobile $m \cdot 2^q$ attraverso la funzione $\texttt{(m, q) = math.frexp(x)}$,

e del valore $s_n = s_{n-1} + c_n p_n$

In [ ]:
def my_sin(point, order, verbose=False):
  s = 0
  p = 1
  coefs = np.array([0,1,0,-1])
  for n in range(1,order+1):

    v_n = (point/n)
    p_prv = p #usato solo per la stampa a video
    p = p * v_n
    Delta = coefs[n%4]*p
    s = s + Delta

    # stampa a video
    if (verbose):
        pprv_str = "p" + str(n-1)
        n_str = str(n)
        p_str = "p" + n_str
        vn_str = "v" + n_str
        msg = ("at " + n_str + ") "
           + pprv_str + ": " + str(math.frexp(p_prv)) + "; "
           + vn_str + ": " + str(math.frexp(v_n))  + " -> "
           + p_str + "=" + pprv_str + "*" + vn_str + ": " + str(p) + " " + str(math.frexp(p))
           + "; s" + n_str + ": " + str(math.frexp(s))
           + " (c" + n_str + ": " + str(coefs[n%4]) + ")"
           )
        print(msg)
  return s


Exercise 1.2 - Solution.

In [ ]:
n = 20
x1 = math.pi/2
x2 = math.pi/2 + 2 * (10 ** 16) * math.pi
#TO DO: complete the script; print the results and comment them.
print("Calcolo di my_sin(x1,n)")
print(my_sin(x1,n))
print("Calcolo di my_sin(x2,n)")
print(my_sin(x2,n,verbose=True))



Calcolo di my_sin(x1,n)
1.0
Calcolo di my_sin(x2,n)
at 1) p0: (0.5, 1); v1: (0.871967124502158, 56) -> p1=p0*v1: 6.2831853071795864e+16 (0.871967124502158, 56); s1: (0.871967124502158, 56) (c1: 1)
at 2) p1: (0.871967124502158, 56); v2: (0.871967124502158, 55) -> p2=p1*v2: 1.9739208802178717e+33 (0.7603266662125618, 111); s2: (0.871967124502158, 56) (c2: 0)
at 3) p2: (0.7603266662125618, 111); v3: (0.581311416334772, 55) -> p3=p2*v3: 4.1341702240399763e+49 (0.8839731424262396, 165); s3: (-0.8839731424262396, 165) (c3: -1)
at 4) p3: (0.8839731424262396, 165); v4: (0.871967124502158, 54) -> p4=p3*v4: 6.49393940226683e+65 (0.7707955191385447, 219); s4: (-0.8839731424262396, 165) (c4: 0)
at 5) p4: (0.7707955191385447, 219); v5: (0.6975736996017263, 54) -> p5=p4*v5: 8.160524927607507e+81 (0.5376866819219079, 273); s5: (0.5376866819219079, 273) (c5: 1)
at 6) p5: (0.5376866819219079, 273); v6: (0.581311416334772, 54) -> p6=p5*v6: 8.545681720669375e+97 (0.6251268132247367, 326); s6: (0.53768668

<ipython-input-32-0c0f464e17eb>:10: RuntimeWarning: invalid value encountered in multiply
  Delta = coefs[n%4]*p


Comment.

Why $sin(x_2) =  \texttt{nan}$ ?

Why does the following error message appear?

```
RuntimeWarning: invalid value encountered in multiply
```



Il motivo del messaggio di errore è dovuto al fatto che nel calcolo del polinomio di Taylor al termine del passo 19 vale
$p_{19} = 0.7017900895422471 \cdot 2^{1004}$
mentre al passo 20 abbiamo

$p_{20} = p_{19} \cdot q_{20}$,

con $q_{20} = 0.7342881048439225 \cdot 2^{52}$

Nella rappresentazione double precision l'esponente può variare tra -1022 e 1023: pertanto quando al passo 20 della sommatoria andiamo a calcolare
$p_{20} = p_{19} \cdot q_{20}$,
abbiamo che la somma degli esponenti (1004 per $p_{19}$, 52 per $q_{20}$)
supera il valore massimo rappresentabile, risultando in $p_{20} = \infty$.

L'operazione successiva prevede poi di moltiplicare $p_{20}=\infty$ per 0 (c[n%4]),
ottenendo NaN

# Exercise 2
Consider the function $f(x) = x^4$ and the following approximations of its first derivative $f'(x)$ via the finite differences
\begin{equation}
\phi_{h,f}^1(x) = \frac{f(x+h)-f(x)}{h}, \quad \phi_{h,f}^2(x) = \frac{f(x+h)-f(x-h)}{2h}.
\end{equation}
1.  Compute the absolute discretization error for $\phi_{h,f}^1$ and $\phi_{h,f}^2$ at $x=1$ for $h=10^{-j}$, $j=1,\dots,12$ and print the results on the video (exponential format).
2.  Comment the results taking into account that, for sufficiently regular functions, the error goes to $0$ for $h\to0$ as $O(h)$ or $O(h^2)$ for $\phi_{h,f}^1$ and $\phi_{h,f}^2$, resepctively.

Exercise 2.1 - Solution.

Segue l'implementazione per $\phi_{h,f}^1(x)$ e $\phi_{h,f}^2(x)$.

Nel corpo principale vengono poi valutati gli scarti rispetto al valore nominale della derivata, senza valore assoluto, al fine di apprezzare anche se la stima fornita è per difetto o in eccesso.

In [1]:
def fun(x):
    return x ** 4
    #return x*x*x*x

def der1_fun(x):
    return 4 * (x ** 3)
    #return 4 * (x*x*x)


def phi1(x, h):
    return (fun(x+h) -fun(x))/h

def phi2(x, h):
    return (fun(x+h) -fun(x-h))/(2*h)


In [ ]:
x = 1
rex = der1_fun(x)
print('h     err1_a   err2_a')
for j in range(1,12+1):
    h = 10 ** (-j)
    r1 = phi1(x, h)
    r2 = phi2(x, h)
    err1_a = r1-rex
    err2_a = r2-rex
    # TODO: print the results
    print(f"{h:.0e} {err1_a:.2e} {err2_a:.2e}")

h     err1_a   err2_a
1e-01 6.41e-01 4.00e-02
1e-02 6.04e-02 4.00e-04
1e-03 6.00e-03 4.00e-06
1e-04 6.00e-04 4.00e-08
1e-05 6.00e-05 3.93e-10
1e-06 6.00e-06 -5.15e-11
1e-07 6.02e-07 1.15e-10
1e-08 2.01e-08 -1.32e-08
1e-09 3.31e-07 1.09e-07
1e-10 3.31e-07 3.31e-07
1e-11 3.31e-07 3.31e-07
1e-12 3.56e-04 1.34e-04


Exercise 2.2 - Solution.

In generale, approssimando tramite Taylor $f(x)$ abbiamo
\begin{equation*}
	f(x+h) = f(x) + f'(x)h + f''(x)\frac{h^2}{2} + f'''(x)\frac{h^3}{6} + o(h^3)
\end{equation*}
Da tale approssimazione ricaviamo
\begin{align*}
	f'(x) &= \frac{f(x+h)-f(x)}{h}    + f''(x)\frac{h}{2} + o(h) \\
	      &= \phi_{h,f}^1(x) + \frac{f''(x)}{2}h + o(h)\\
	f'(x) &= \frac{f(x+h)-f(x-h)}{2h} + f'''(x)\frac{h^2}{6} + o(h^2)\\
	      &= \phi_{h,f}^2(x) + \frac{f'''(x)}{6}h^2 + o(h^2)
\end{align*}
da cui abbiamo che l'errore di troncamento $\delta_{f,k}(x,h) = f'(x) - \phi_{h,f}^k(x)$ per $k \in \{1,2\}$, generato dall'approssimazione con le differenze finite, vale
\begin{align*}
	\delta_{f,1}(x,h) &= \frac{f''(x)}{2}h + o(h) \approx \frac{f''(x)}{2}h\\
	\delta_{f,2}(x,h) &= \frac{f'''(x)}{6}h^2 + o(h^2) \approx \frac{f'''(x)}{6}h^2
\end{align*}

Pertanto in $x=1$ per $f(x)=x^4$ (inoltre $f'(1)=4$, $f''(1)=12$, $f'''(1)=24$), otteniamo che gli errori dovuti all'approssimazione della derivata tramite differenze finite sono descrivibili come
\begin{align*}
	\delta_{f,1}(1,h) &= 6h\\
	\delta_{f,2}(1,h) &= 4h^2
\end{align*}


I valori forniti da err$k$\_a rappresentano invece le versioni numeriche per $\delta_{f,k}(1,h)$ in caso di precisione finita: essi concordano parzialmente con i risultati esatti, fino ad un certo valore di $h$, dopodiché intervengono errori di arrotodondamento causati dalla rappresentazione finita dei numeri macchina.

Comment.

Is the error behaviour aligned with the theoretical results?
If not, why?

In particolare per i valori precedentemente tabulati l'equivalenza smette di valere per $h=10^{-j}$ quando (abbuoniamo i casi in cui la discrepanza è piccola):
* $j\ge 8$  (per $k=1$);
* $j\ge 6$  (per $k=2$).

Senza perdere di generalità analizziamo gli errori di arrotondamento dovuti a $\phi_{h,f}^1(1)$.

Ricordando che in double-precision $\epsilon_m = 2^{-52} \approx 2.2204 \cdot 10^{-16}$, il problema principale sorge nel calcolo di $(1+h)^4 - 1$; per semplicità consideriamo il caso più eclatante ($h=10^{-12}$):
\begin{equation*}
	\text{fl}\big((1 + h)^4\big) = \text{fl}(1 + 4h + 6h^2 +4h^3 + h^4) = 1 + 4h \pm \alpha\epsilon_m
\end{equation*}
in quanto $h^4 < 4h^3 < 6h^2 < \epsilon_m$ ed il contributo $\pm \alpha\epsilon_m$ tiene conto dei ``buchi'' nella rappresentazione floating point.
In effetti abbiamo che
* $\texttt{math.frexp((1+h)**4) = (0.5000000000020002, 1)}$ (1.0000000000040004)
* $\texttt{math.frexp((1+4*h))   = (0.500000000002, 1)}$ (1.000000000004)
* $\texttt{(1+h)**4-1 = 4.000355602329364e-12}$

Trascurando la propagazione degli errori successivi, meno impattanti, abbiamo
\begin{equation*}
	\frac{4h \pm \alpha\epsilon_m}{h} - 4 = (4 \pm \alpha \epsilon_m h^{-1}) -4 = \pm \alpha \epsilon_m \cdot 10^{12} \approx \pm \alpha 2.2204 \cdot 10^{-4}
\end{equation*}
che è in linea con l'ordine di grandezza del valore numerico ottenuto.

Ragionamenti simili sono applicabili anche agli altri valori di $h$ e nel caso di $\phi_{h,f}^2(x)$.

In generale, il valore ottimo per $h$ deve essere un compromesso tra rendere $h$ il più piccolo possibile in modo da ridurre l'errore $\delta_{f,k}(x,h)$, senza però abbassarlo eccessivamente per non far emergere gli errori di arrondamento: ad esempio, per avere un'idea dell'ordine di grandezza sotto il quale non scendere per $h$, nel caso di $\phi_{h,f}^1(1)$ per $f(x)=x^4$ se poniamo come condizione limite
\begin{equation}
\frac{f(x+h)-f(x)}{h} \approx \frac{\epsilon_m}{h}
\end{equation}
e la equipariamo a $\delta_{f,1}(1,h) = 6h$ otteniamo
\begin{equation}
    6h = \frac{\epsilon_m}{h} \Rightarrow h = \sqrt{\frac{\epsilon_m}{6}} = \left( \frac{2^{-52}}{6} \right)^{1/2} =
    \sqrt{1/6} \cdot 2^{-26} \approx 6 \cdot 10^{-9}
\end{equation}
La stima di tale limite trova riscontro nei valori di $\phi_{h,f}^1(1)$ precedentemente tabulati (per $h=10^{-8}$ dobbiamo tener conto che $\text{fl}\big((1+h)^4\big) = 1+ 4h + 6h^2 \pm \alpha \epsilon$, in quanto in tal caso $6h^2 > \epsilon_m$ ma dello stesso ordine di grandezza).


Nel caso di $\phi_{h,f}^2(1)$ per $f(x)=x^4$, ponendo $\delta_{f,2}(1,h) = \epsilon_m/h$ otteniamo come limite per h
\begin{equation}
\delta_{f,2}(1,h) = 4h^2 = \frac{\epsilon_m}{h} \Rightarrow h = \sqrt[3]{\epsilon_m/4} = 2^{-18} \approx 3.8 \cdot 10^{-6}
\end{equation}
anch'esso in accordo ai valori tabulati.

# Exercise 3
Consider the following second order equation
\begin{equation}
  x^2 - 2x + \delta = 0.
\end{equation}
The solutions are
\begin{equation}
  x_1 = 1 + \sqrt{1-\delta}\quad \text{and}\quad x_2 = 1 - \sqrt{1-\delta}.  
\end{equation}

1.  Compute $x_1$ and $x_2$ for different input $\delta$ values.
2.  Using the function(s) of point (1), evaluate $x_1$ and $x_2$ for  $\delta = 10^{-1}, 10^{-3}, 10^{-8}$.
Compare and comment the results with the ones obtained using the predefinite numpy function ```np.roots(p)``` , which returns the roots of a polynomial with coefficients given in `p`.
3. Is it possible to improve the results using a different formula to compute $x_2$?



Exercise 3.1 - Solution.

In [ ]:
def solution_1(delta):
  return 1 + math.sqrt(1 - delta)

def solution_2(delta):
  return 1 - math.sqrt(1 - delta)

Exercise 3.2 - Solution

In [ ]:
print('delta err1_1   err2_r')
for i in [-1, -3, -8]:
  delta = 10 ** i
  sol = np.roots(np.array([1, -2, delta]))
  x1  = max(sol[0],sol[1]) #valore maggiore
  x2  = min(sol[0],sol[1]) #valore minore

  my_x1 = solution_1(delta)
  my_x2 = solution_2(delta)
  err1_r = abs(x1-my_x1)/abs(x1)#TO DO: compute error
  err2_r = abs(x2-my_x2)/abs(x2)#TO DO: compute error
  # TO DO: print the results
  print(f"{delta:.0e} {err1_r:.2e} {err2_r:.2e}")

delta err1_1   err2_r
1e-01 0.00e+00 5.41e-16
1e-03 0.00e+00 8.50e-14
1e-08 0.00e+00 1.36e-08


Comment.

What happens to $x_2$?

Il problema è dovuto alla sottrazione tra 1 e $\sqrt{1-\delta}$ che causa errori di arrotondamento/propagazione crescenti al diminuire di $\delta$

Exercise 3.3 - Solution.

Hint: given
\begin{equation}
  ax^2 + bx + c = 0
\end{equation}
let $x_1$ and $x_2$ be its roots. Hence,
\begin{split}
  &x_1 + x_2 = -\frac{b}{a},\\
  &x_1 * x_2 = \frac{c}{a}.
\end{split}

In our case $a=1$, $b=-2$, $c=\delta$

\begin{split}
  &x_1 + x_2 = 2,\\
  &x_1 * x_2 = \delta.
\end{split}

Sfruttando le relazioni indicate, conviene calcolare la soluzione come
* $x_1 = 1 + \sqrt{1+\delta}$ (ossia la soluzione meno perturbata)
* $x_2 = \delta/x_1$

La formula per $x_2$ è preferibile a $x_2=2-x_1$ perché altrimenti avrei una sottrazione tra valori vicini tra loro ($x_1 \approx 2$)

In [ ]:
print('delta err1_1   err2_r')
for i in [-1, -3, -8]:
  delta = 10 ** i
  sol = np.roots(np.array([1, -2, delta]))
  x1  = max(sol[0],sol[1]) #valore maggiore
  x2  = min(sol[0],sol[1]) #valore minore

  my_x1 = solution_1(delta) #TO DO: compute x1
  my_x2 = delta/my_x1 #TO DO: compute x2
  err1_r = abs(x1-my_x1)/abs(x1)#TO DO: compute error
  err2_r = abs(x2-my_x2)/abs(x2)#TO DO: compute error

  print(f"{delta:.0e} {err1_r:.2e} {err2_r:.2e}")

delta err1_1   err2_r
1e-01 0.00e+00 0.00e+00
1e-03 0.00e+00 0.00e+00
1e-08 0.00e+00 0.00e+00
